<a href="https://colab.research.google.com/github/filchyboy/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS11_212_Regression_2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [X] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

#Code & Param Setup

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

#Do a train test split

In [0]:
import sklearn as sk
from sklearn.model_selection import train_test_split

In [0]:
split = train_test_split(df, shuffle=False)
train = split[0]
test = split[1]

In [6]:
print(split[0].shape)
print(split[1].shape)

(36612, 34)
(12205, 34)


#Engineer Features


##Cats or Dogs Feature

In [7]:
train['dog_or_cat'] = train['cats_allowed'] + train['dogs_allowed']
test['dog_or_cat'] = test['cats_allowed'] + test['dogs_allowed']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
print(train.head())
print(train.shape)
print(test.head())
print(test.shape)

   bathrooms  bedrooms  ... common_outdoor_space dog_or_cat
0        1.5         3  ...                    0          0
1        1.0         2  ...                    0          2
2        1.0         1  ...                    0          0
3        1.0         1  ...                    0          0
4        1.0         4  ...                    0          0

[5 rows x 35 columns]
(36612, 35)
       bathrooms  bedrooms  ... common_outdoor_space dog_or_cat
37024        1.0         1  ...                    0          0
37025        2.0         2  ...                    0          2
37026        2.0         2  ...                    0          2
37027        1.0         1  ...                    0          0
37028        2.0         3  ...                    0          2

[5 rows x 35 columns]
(12205, 35)


##Number of Rooms Feature

In [9]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
train['no_rooms'] = train['bathrooms'] + train['bedrooms'] + train['dining_room']
test['no_rooms'] = test['bathrooms'] + test['bedrooms'] + test['dining_room']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
print(train.shape)
print(test.shape)

(36612, 36)
(12205, 36)


#Linear Regression on 2 Features

In [12]:
y_train = train['price'].mean()
print('Train Mean:',y_train)
y_test = test['price'].mean()
print('Test Mean:',y_test)

Train Mean: 3584.2734622528133
Test Mean: 3565.5217533797622


In [13]:
px.scatter(train, x='price', y='dog_or_cat', trendline='ols')

In [14]:
px.scatter(test, x='price', y='dog_or_cat', trendline='ols')

In [0]:
regression = LinearRegression()
features = ['cats_allowed','dogs_allowed']
target = 'price'

X_train = train[features]
y_train = train[target]


In [16]:
print('X_train:',X_train)
print('y_train:',y_train)

X_train:        cats_allowed  dogs_allowed
0                 0             0
1                 1             1
2                 0             0
3                 0             0
4                 0             0
...             ...           ...
37019             0             0
37020             0             0
37021             1             0
37022             1             1
37023             0             0

[36612 rows x 2 columns]
y_train: 0        3000
1        5465
2        2850
3        3275
4        3350
         ... 
37019    8500
37020    3795
37021    2650
37022    7395
37023    3000
Name: price, Length: 36612, dtype: int64


In [17]:
regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
print(regression.coef_)
print(regression.intercept_)

[-135.1277146   334.34080237]
3498.4665445325677


###Sorting out some confusion on my part between series and dataframes.

In [19]:
type(train['price'])

pandas.core.series.Series

In [20]:
type(train[['cats_allowed']])

pandas.core.frame.DataFrame

In [21]:
type(train[['dogs_allowed']])

pandas.core.frame.DataFrame

In [22]:
train['price'].shape

(36612,)

In [23]:
train[['dogs_allowed']].shape

(36612, 1)

In [24]:
train[['cats_allowed']].shape

(36612, 1)

In [25]:
train[['dogs_allowed']].head()

,dogs_allowed
0,0
1,1
2,0
3,0
4,0


In [26]:
train[['cats_allowed']].head()

,cats_allowed
0,0
1,1
2,0
3,0
4,0


##Begin Prediction Testing

In [27]:
x_var = test[features]
X_test = [[x_var]]
y_pred = regression.predict(X_test)
y_pred

ValueError: ignored

In [0]:
mae = mean_absolute_error(y_test, y_pred)
print(mae)
estimate = y_pred[0]
coefficient = regression.coef_[0]
print(estimate)
print(coefficient)
